In [27]:
import os
import pandas as pd
import swifter
import spacy
from spacy.lang.en import English
import pickle
from tqdm import tqdm
import nltk
import string
import json

from rank_bm25 import BM25Okapi as BM25
from sentence_transformers import SentenceTransformer, util

In [28]:
nlp = English()

wn = nltk.WordNetLemmatizer()

In [29]:
class caseEntailment():

    def __init__(self, datasetName='COLIEE2021', stopwords_file='stopwords.txt', resultFile='Results.txt', python36=False):
					
        self.stopwords_file = stopwords_file
        self.stopwordLst = []
        self.resultFile = resultFile
        self.datasetName = datasetName

        if(datasetName=='COLIEE2021'):
            self.test_folder = datasetName+"/task_2/task2_2021_test_nolabels"
            self.train_folder = datasetName+"/task_2/task2_2021_train"
            self.test_labels = datasetName+"/task_2/task2_test_labels_2021.json"
            self.train_labels = datasetName+"/task_2/task2_train_labels_2021.json"
        elif(datasetName=='COLIEE2022'):
            self.test_folder = datasetName+"/task_2/task2_2022_test"
            self.train_folder = datasetName+"/task_2/task2_2022_train"
            self.test_labels = datasetName+"/task_2/task2_test_labels_2022.json"
            self.train_labels = datasetName+"/task_2/task2_train_labels_2022.json"
        else:
            raise Exception("Invalid dataset name")
	
        self.caseDataFrame = None
        self.caseDataFrame_train = None

        self.bm25 = None 
        self.model = None
        self.customStopwordLst = [" " , "\n", "'s", "...", "\n ", " \n", " \n ", "\xa0"]

        if(python36):
			# in case we ever need to use a server that has a lower python version, not in use currently
            pass
        else:
            self.pickleFilePath = datasetName+"/task_2/test_pickles/clean_query_coliee2021"
            self.pickleFilePath_train = datasetName+"/task_2/train_pickles/clean_query_coliee2021"


########## Clean Data

	# creates a list of stopwords
    @staticmethod
    def read_and_parse_stopwords_file(self):

        # Open file from string 
        file = open(self.stopwords_file, "r")

        # Read from provided stopwords_file file
        raw_data_stopwords = file.read()

        # Assign list of stopwords_file
        self.stopwordLst = raw_data_stopwords.replace('\t', '\n').split('\n')

        file.close()

        pass

	# tokenize and lemmatize words
    @staticmethod
    def cleanText(self,txt):

        my_doc = nlp(txt)
        tokens = [token.text for token in my_doc]

        lst = []
        for word in tokens:
            low = word.lower()
            lem = wn.lemmatize(low)
            if (lem not in self.stopwordLst) and (lem not in self.customStopwordLst) and (lem not in list(string.punctuation)):
                lst.append(lem)
			
        return lst


########## Initialize Data
	
	# Creates a pandas DataFrame with columns: case_number, paragraphs, paragraph_names, base_case, entailed_fragment
	# it also contains a "clean" version of some of the columns: base_case_clean, entailed_fragment_clean, paragraphs_clean						
    @staticmethod
    def dataFrameCreation(self, folderCategory, dataframe, pickleFilePath):

        folder_names = []
        paragraph_per_folder = []
        paragraph_names_per_folder = []
        query_cases = []
        decisions = []

		# parse through each case
        for folder_name in os.listdir(folderCategory):
            folder = os.path.join(folderCategory, folder_name)
            paragraphs = []
            paragraph_names = []
            # create list of paragraph names and paragraph content of the specific case
            for paragraph_name in os.listdir(folder+'/paragraphs'):
		
                f = open(os.path.join(folder+'/paragraphs', paragraph_name))
                paragraphs.append(f.read())
                paragraph_names.append(paragraph_name.replace(".txt",""))

            # extract the base case and the decision of the specific case
            base_case = open(folder+'/base_case.txt')
            decision = open(folder+'/entailed_fragment.txt')

            folder_names.append(folder_name)
            paragraph_per_folder.append(paragraphs)
            paragraph_names_per_folder.append(paragraph_names)
            query_cases.append(base_case.read())
            decisions.append(decision.read())

        dataframe = pd.DataFrame({'case_number': folder_names, 'paragraphs': paragraph_per_folder, 'paragraph_names': paragraph_names_per_folder, 'base_case': query_cases, 'entailed_fragment': decisions})								

        dataframe['base_case_clean'] = dataframe['base_case'].swifter.apply(lambda x: self.cleanText(self, x))

        dataframe['entailed_fragment_clean'] = dataframe['entailed_fragment'].swifter.apply(lambda x: self.cleanText(self, x))

        paragraphs_clean = []
        for paras in paragraph_per_folder:
            l=[]
            for p in paras:
                p_clean = self.cleanText(self, p)
                l.append(p_clean)
            paragraphs_clean.append(l)

        dataframe['paragraphs_clean'] = paragraphs_clean

        dataframe.to_pickle(pickleFilePath)

	# create test and train DataFrame
    def createDataFrames(self):
	
        self.read_and_parse_stopwords_file(self)
        self.dataFrameCreation(self, self.test_folder, self.caseDataFrame, self.pickleFilePath)
        self.dataFrameCreation(self, self.train_folder, self.caseDataFrame_train, self.pickleFilePath_train)

	# Once DataFrames have been created, they can be retreived using this function
    def preProcess(self):
        self.caseDataFrame = pd.read_pickle(self.pickleFilePath)
        self.caseDataFrame_train = pd.read_pickle(self.pickleFilePath_train)


########## Evaluate Data

    @staticmethod
    def calculateRecall(self, results, labels, topn):
        lf = open(labels, "r")
        rf = open(results, "r")

        results_lines = rf.readlines()
        labels_json = json.load(lf)
        rel_num = 0
        rel_cases = 0
        for label in labels_json.keys():
            relevant_docs = labels_json[label].split(", ")
            rel_cases += len(relevant_docs)
            for i in range(len(relevant_docs)):
                relevant_docs[i] = relevant_docs[i].replace(".txt","")
            retreived = []
            for line in results_lines:
                line_split = line.split('\t')
                if(line_split[0]==label.replace(".txt","")):
                    retreived.append(line_split[2])

            for i in range(min(topn, len(retreived))):
                if(retreived[i] in relevant_docs):
                    rel_num+=1

        recall = rel_num/rel_cases
        print("recall at top "+str(topn)+" found is", recall)
        return recall

    @staticmethod
    def calculatePrecision(self, results, labels, topn):
        lf = open(labels, "r")
        rf = open(results, "r")
        results_lines = rf.readlines()
        labels_json = json.load(lf)
        rel_num = 0
        num_queries=0
        for label in labels_json.keys():
            num_queries +=1
            relevant_docs = labels_json[label].split(", ")
            for i in range(len(relevant_docs)):
                relevant_docs[i] = relevant_docs[i].replace(".txt","")
            retreived = []
            for line in results_lines:
                line_split = line.split('\t')
                if(line_split[0]==label.replace(".txt","")):
                    retreived.append(line_split[2])
		
            for i in range(min(topn, len(retreived))):
                if(retreived[i] in relevant_docs):
                    rel_num+=1

        retreived_cases = num_queries*topn
        precision = rel_num/retreived_cases
        print("precision at top "+str(topn)+" found is", precision)
        return precision



    def calculateF1(self, results, labels, topn):
        recall = self.calculateRecall(self, results, labels, topn)
        precision = self.calculatePrecision(self, results, labels, topn)
        f1 = (2*precision*recall)/(precision+recall)
        print("f1 at top "+str(topn)+" found is", f1)
        return f1


########## Computation


	# Write output in result txt file
    @staticmethod
    def results(testQuerieNum, rankedDocs, resultFile, topn=100):

        for x in range(min(topn, len(rankedDocs))):
            rank = str(x + 1)
            docID, score = rankedDocs[x]
            resultFile.write(testQuerieNum + "\tQ0\t" + str(docID) +
                    "\t" + rank + "\t" + str(score) + "\tmyRun\n")
            pass
        pass 




	#returns ranked by bm25 score dictionary with doc id as key and score as value
    @staticmethod
    def rankDocsBM25(self, testQuerie, documentLabels):
        # calculate scores
        doc_scores = self.bm25.get_scores(testQuerie)

        # assign scores to document label
        x=0
        listScores = []
        for sim in doc_scores:
            listScores.append((documentLabels[x], sim))
            x += 1

        # create list of tuples with first element as doc id and second as value
        rankedDocs = [(k, v) for k, v in sorted(listScores, key=lambda item: item[1], reverse=True)]
        return rankedDocs

	# Use BM25 score to measure entailement
    def bm25Entailment(self):
        results = open(self.resultFile, 'w+')
        totalCases = len(self.caseDataFrame)
        for caseNum in tqdm(range(totalCases)):
            # initialize bm25 with paragraphs
            self.bm25 = BM25(self.caseDataFrame['paragraphs_clean'][caseNum])
            # calculate scores
            rankedDocs = self.rankDocsBM25(self, self.caseDataFrame['entailed_fragment_clean'][caseNum], self.caseDataFrame['paragraph_names'][caseNum])

            # write results
            self.results(self.caseDataFrame['case_number'][caseNum], rankedDocs, results, topn=5)

        results.close()
        
         
    def transformer_preprocess(self, model_name):

        #load model
        self.model = SentenceTransformer(model_name)

        # check if we can use GPU for training
        #         device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        #         torch.cuda.empty_cache()
        #         self.model.to(device)
        return self.model

#     @staticmethod
    def FindSimilarityWithTransformer(self, entailed_fragment, paragraph):

        # create embeddings of our two sentences
        query_embedding = self.model.encode(entailed_fragment, normalize_embeddings=True)
        passage_embedding = self.model.encode(paragraph, normalize_embeddings=True)

        # find their cosine similarity by calculating dot score of their embeddings
        similarity = util.dot_score(query_embedding, passage_embedding)[0][0]

        return similarity.item()
    

    
    # evaluate similarity for sentences with given model
    def EvaluateSimilaritySBERT(self):
        results = open(self.resultFile, 'w+')
        self.model = self.transformer_preprocess("bert-base-nli-mean-tokens")

        # parse through our 5 datasets of sentence pairs
        totalCases = len(self.caseDataFrame)
        for caseNum in tqdm(range(totalCases)):

            similarity_scores = []
            print("Processing:", self.caseDataFrame['case_number'][caseNum])

            # parse through each sentence pair of dataset
            for i in tqdm(range(len(self.caseDataFrame['paragraphs'][caseNum]))):
                # calculate their cosine similarity
                similarity_scores.append((self.caseDataFrame['paragraph_names'][caseNum][i],self.FindSimilarityWithTransformer(self.caseDataFrame['entailed_fragment'][caseNum], self.caseDataFrame['paragraphs'][caseNum][i])))

            sortedDocs = [(k, v) for k, v in sorted(similarity_scores, key=lambda item: item[1], reverse=True)]
            self.results(self.caseDataFrame['case_number'][caseNum], sortedDocs, results, topn=5)

        results.close()
        
        
        

In [30]:
entailment_model = caseEntailment('COLIEE2021')

In [31]:
entailment_model.preProcess()

In [32]:
entailment_model.bm25Entailment()

100%|██████████| 100/100 [00:00<00:00, 782.29it/s]


In [33]:
entailment_model.EvaluateSimilaritySBERT()

  0%|          | 0/30 [00:00<?, ?it/s]

Processing: 468



  0%|          | 0/242 [00:00<?, ?it/s]

Processing: 446



  0%|          | 0/40 [00:00<?, ?it/s]

...

Processing: 454



  0%|          | 0/46 [00:00<?, ?it/s]

Processing: 449



 55%|█████▌    | 55/100 [12:42<10:24, 13.87s/it]


KeyboardInterrupt: 

In [19]:
entailment_model.calculateF1(entailment_model.resultFile, entailment_model.test_labels, 1)

recall at top 1 found is 0.5641025641025641
precision at top 1 found is 0.66
f1 at top 1 found is 0.6082949308755762


0.6082949308755762

In [40]:
entailment_model.caseDataFrame_train.columns

Index(['case_number', 'paragraphs', 'paragraph_names', 'base_case',
       'entailed_fragment', 'base_case_clean', 'entailed_fragment_clean',
       'paragraphs_clean'],
      dtype='object')

In [35]:
entailment_model.caseDataFrame_train['paragraphs'][0][2]

'[24]\nIn the case at bar, the applicant presented as part of his new evidence a statement made by Robert Stratton, a fellow member of his unit, which supports the applicant\'s claim that he was injured in a truck accident while on duty. In a "Statement by Witness" form from the Bureau of Pensions Advocates, Mr. Stratton answers in the affirmative to the question: "Did the applicant ever discuss his physical condition with you?" Asked to provide particulars, he writes,\n"Yes–following an accident and injury I saw him sustain while we were riding together in the back of a truck at the Bedford Rifle Range. We were both at that time members of Halifax Rifles Militia. He complained of back pain." (AR, tab 4)'

In [36]:
entailment_model.caseDataFrame_train['paragraph_names'][0][2]

'024'

In [38]:
entailment_model.caseDataFrame_train['entailed_fragment'][0]

'the applicable standard of review of decisions of the Veterans Review Appeal Board is that of patent unreasonableness'

In [ ]:
  #
def transformer_preprocess(self, model_name):

    #load model
    model = SentenceTransformer(model_name)

    # check if we can use GPU for training
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    torch.cuda.empty_cache()
    model.to(device)

    return model

In [ ]:
def FindSimilarityWithTransformer(df,sentence_pair, dataset_num, model, datasets):

   



    # create embeddigns of our two sentences
    query_embedding = model.encode(sentence_pair[0], normalize_embeddings=True)
    passage_embedding = model.encode(sentence_pair[1], normalize_embeddings=True)

    # find their cosine similarity by calculating dot score of their embeddings
    similarity = util.dot_score(query_embedding, passage_embedding)[0][0]

    return similarity.item()




In [ ]:
# evaluate similarity for sentences with given model
def EvaluateSimilarity(sentences, datasets, model_name):

    model = transformer_preprocess('bert-base-nli-mean-tokens')

    # parse through our 5 datasets of sentence pairs
    for dataset_num in range(len(datasets)):
        similarity_scores = []
        print("Processing:", datasets[dataset_num])
        # parse through each sentence pair of dataset
        for i in tqdm(range(len(sentences[dataset_num]))):
        
        ####
        
        totalCases = len(df)
        for caseNum in tqdm(range(totalCases)):
            # initialize bm25 with paragraphs
            (df['paragraphs'][caseNum])
            
            df['paragraphs'][caseNum]
            
            # calculate scores
            (df['entailed_fragment'][caseNum], df['paragraph_names'][caseNum])
            # write results
            self.results(self.caseDataFrame['case_number'][caseNum], rankedDocs, results, topn=5)
            # calculate their cosine similarity
            
        ####
        
                case "Roberta":
                    similarity_scores.append(str(FindSimilarityWithTransformer(sentences[dataset_num][i], dataset_num, model, datasets)))

        # write down our results in txt file
        with open("./data/sts2016-english-with-gs-v1.0/SYSTEM_OUT."+datasets[dataset_num]+".txt", 'w') as f:
            for sim in similarity_scores:
                f.write(sim)
                f.write("\n")